# PIPELINE CREATION, RETRAINING AND EXECUTION SCRIPTS

## PIPELINE CREATION

In [5]:
import os
import cloudpickle
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

# Paths
MODELS_PATH = '/Users/rober/smartport-ai-risk-early-warning/04_Models/'
if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)

# Build Skeleton (The blank form)
pipe_retraining = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Save
with open(os.path.join(MODELS_PATH, 'pipe_retraining.pkl'), 'wb') as f:
    cloudpickle.dump(pipe_retraining, f)

print("✔ pipe_retraining.pkl created")

✔ pipe_retraining.pkl created


In [1]:
import os
import cloudpickle
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

# Project Paths
PROJECT_PATH = '/Users/rober/smartport-ai-risk-early-warning'
MODELS_PATH = os.path.join(PROJECT_PATH, '04_Models')

if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)

# Build the Production Skeleton using XGBoost
# We use an imputer just in case new real-time data contains NaNs
pipe_retraining = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('model', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])

# Save the skeleton
with open(os.path.join(MODELS_PATH, 'pipe_retraining.pkl'), 'wb') as f:
    cloudpickle.dump(pipe_retraining, f)

print("✔ Success: pipe_retraining.pkl (XGBoost) created.")

✔ Success: pipe_retraining.pkl (XGBoost) created.


## RETRAINING SCRIPT

In [6]:
import os
import pandas as pd
import cloudpickle
from sklearn.metrics import recall_score

# Paths
BASE_PATH = '/Users/rober/smartport-ai-risk-early-warning/'
TRAIN_DATA = os.path.join(BASE_PATH, '02_Data/03_Working/work_fs.csv')
SKELETON = os.path.join(BASE_PATH, '04_Models/pipe_retraining.pkl')
EXECUTION_PIPE = os.path.join(BASE_PATH, '04_Models/pipe_execution.pkl')

def run_retraining():
    # Load and Clean
    df = pd.read_csv(TRAIN_DATA)
    
    # Validation: Drop rows where target is missing (ML cannot learn from NaN targets)
    df = df.dropna(subset=['delay_flag'])
    
    X = df.drop(columns=['delay_flag'])
    y = df['delay_flag']
    
    # Load Skeleton
    with open(SKELETON, 'rb') as f:
        pipe = cloudpickle.load(f)
    
    # Train
    pipe.fit(X, y)
    
    # Audit (Threshold 0.02)
    probs = pipe.predict_proba(X)[:, 1]
    recall = recall_score(y, (probs >= 0.02).astype(int))
    
    if recall >= 0.95:
        with open(EXECUTION_PIPE, 'wb') as f:
            cloudpickle.dump(pipe, f)
        print(f"✔ retraining.py: Model promoted with Recall: {recall:.4f}")
    else:
        print(f"✘ retraining.py: Recall {recall:.4f} below 0.95 limit.")

if __name__ == "__main__":
    run_retraining()

✔ retraining.py: Model promoted with Recall: 1.0000


In [3]:
# ==============================================================================
# 2. AUTOMATED RETRAINING SCRIPT (CORRECTED)
# ==============================================================================
import os
import pandas as pd
import cloudpickle
from imblearn.combine import SMOTETomek
from sklearn.metrics import recall_score
from sklearn.impute import SimpleImputer

def run_retraining():
    BASE_PATH = '/Users/rober/smartport-ai-risk-early-warning'
    TRAIN_DATA = os.path.join(BASE_PATH, '02_Data/03_Working/work_fs.csv')
    SKELETON = os.path.join(BASE_PATH, '04_Models/pipe_retraining.pkl')
    EXECUTION_PIPE = os.path.join(BASE_PATH, '04_Models/pipe_execution.pkl')

    # 1. Load Data
    df = pd.read_csv(TRAIN_DATA).dropna(subset=['delay_flag'])
    X = df.drop(columns=['delay_flag'])
    y = df['delay_flag']
    
    # 2. FIX: Handle NaNs BEFORE SMOTE-Tomek
    # We must impute values now because SMOTE needs a complete matrix to calculate distances
    imputer = SimpleImputer(strategy='median')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    # 3. Apply Class Balancing (SMOTE-Tomek)
    print("Balancing classes with SMOTE-Tomek...")
    smt = SMOTETomek(random_state=42)
    X_res, y_res = smt.fit_resample(X_imputed, y)
    
    # 4. Load Skeleton and Train
    with open(SKELETON, 'rb') as f:
        pipe = cloudpickle.load(f)
    
    # Train
    pipe.fit(X_res, y_res)
    
    # 5. Audit: Ensure the model catches at least 90% of delays
    probs = pipe.predict_proba(X_res)[:, 1]
    recall = recall_score(y_res, (probs >= 0.5).astype(int))
    
    if recall >= 0.90:
        with open(EXECUTION_PIPE, 'wb') as f:
            cloudpickle.dump(pipe, f)
        print(f"✔ Retraining successful. Model promoted with Recall: {recall:.4f}")
    else:
        print(f"✘ Retraining failed. Recall {recall:.4f} is too low.")

if __name__ == "__main__":
    run_retraining()

Balancing classes with SMOTE-Tomek...


/Users/rober/opt/miniconda3/envs/smartport/lib/python3.10/site-packages/xgboost/training.py:199: UserWarning: [12:20:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✔ Retraining successful. Model promoted with Recall: 0.9625


## EXECUTION SCRIPT

In [9]:
import os
import pandas as pd
import cloudpickle
from datetime import datetime

# Paths
BASE_PATH = '/Users/rober/smartport-ai-risk-early-warning/'
MODEL_FILE = os.path.join(BASE_PATH, '04_Models/pipe_execution.pkl')
SOURCE_DATA = os.path.join(BASE_PATH, '02_Data/03_Working/work_fs.csv')
OUTPUT_CSV = os.path.join(BASE_PATH, '05_Outputs/predictions.csv')

# Define Actionable Business Logic
DECISION_MAP = {
    'CRITICAL': 'IMMEDIATE: Reassign Docking Slot & Notify Tugboats',
    'WARNING': 'PROACTIVE: Request GPS/ETA update from Vessel',
    'NORMAL': 'ROUTINE: Maintain standard schedule'
}

def run_execution():
    if not os.path.exists(MODEL_FILE):
        print("✘ Error: pipe_execution.pkl not found.")
        return

    with open(MODEL_FILE, 'rb') as f:
        model = cloudpickle.load(f)
    
    df = pd.read_csv(SOURCE_DATA)
    
    # 1. Prepare Features
    X_live = df.drop(columns=['delay_flag']) if 'delay_flag' in df.columns else df
    
    # 2. Get Risk Scores
    probs = model.predict_proba(X_live)[:, 1]
    
    # 3. Create Results with Decision Layer
    results = pd.DataFrame()
    results['vessel_id'] = df.index 
    results['risk_score'] = probs
    
    # Calibrating thresholds to handle the high average risk score (0.85)
    # This ensures operational relevance by only flagging extreme outliers
    results['risk_level'] = results['risk_score'].apply(
        lambda x: 'CRITICAL' if x >= 0.99 else (   # Top tier risk
                   'WARNING' if x >= 0.95 else     # Significant risk above average
                   'NORMAL')                       # Baseline port operations
    )
    
    # Updating actions to match professional logistics standards
    results['recommended_action'] = results['risk_level'].map(DECISION_MAP)
    
    results['execution_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # 4. Save
    results.to_csv(OUTPUT_CSV, index=False)
    print(f"✔ execution.py: Decisions saved in {OUTPUT_CSV}")

    # Terminal summary to verify balance
    print("\n--- Operational Summary ---")
    print(results['risk_level'].value_counts())
    print(f"\nAverage Risk Score: {results['risk_score'].mean():.4f}")

if __name__ == "__main__":
    run_execution()

✔ execution.py: Decisions saved in /Users/rober/smartport-ai-risk-early-warning/05_Outputs/predictions.csv

--- Operational Summary ---
risk_level
CRITICAL    64617
NORMAL      39526
WARNING     12338
Name: count, dtype: int64

Average Risk Score: 0.8592


In [4]:
import os
import pandas as pd
import cloudpickle
from datetime import datetime

# Business Decisions based on Risk Levels
ACTION_MAP = {
    'CRITICAL': 'IMMEDIATE: Priority berthing & Tugboat standby.',
    'WARNING': 'PROACTIVE: Verify ETA and terminal capacity.',
    'NORMAL': 'ROUTINE: Follow standard operations.'
}

def run_execution():
    BASE_PATH = '/Users/rober/smartport-ai-risk-early-warning'
    MODEL_FILE = os.path.join(BASE_PATH, '04_Models/pipe_execution.pkl')
    SOURCE_DATA = os.path.join(BASE_PATH, '02_Data/03_Working/work_fs.csv') # Simulating new data
    OUTPUT_CSV = os.path.join(BASE_PATH, '05_Outputs/risk_alerts.csv')

    if not os.path.exists(MODEL_FILE):
        print("✘ Error: Execution model not found. Run retraining first.")
        return

    # 1. Load Model and Data
    with open(MODEL_FILE, 'rb') as f:
        model = cloudpickle.load(f)
    
    df = pd.read_csv(SOURCE_DATA)
    X_live = df.drop(columns=['delay_flag']) if 'delay_flag' in df.columns else df
    
    # 2. Scoring
    probs = model.predict_proba(X_live)[:, 1]
    
    # 3. Alert Logic
    results = pd.DataFrame({
        'vessel_index': df.index,
        'risk_score': probs
    })
    
    # Thresholds calibrated for balanced XGBoost
    results['risk_level'] = results['risk_score'].apply(
        lambda x: 'CRITICAL' if x >= 0.90 else (
                   'WARNING' if x >= 0.70 else 'NORMAL')
    )
    results['recommended_action'] = results['risk_level'].map(ACTION_MAP)
    results['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # 4. Export
    results.to_csv(OUTPUT_CSV, index=False)
    print(f"✔ Execution completed. Alerts saved in {OUTPUT_CSV}")

if __name__ == "__main__":
    run_execution()

✔ Execution completed. Alerts saved in /Users/rober/smartport-ai-risk-early-warning/05_Outputs/risk_alerts.csv


The automated retraining script has successfully promoted a new version of the model. 

Recall = 0.9625
* The model is extremely sensitive to delay signals. Out of 100 potential delays, the system correctly flags 96 of them.
* The high recall ensures that the Port Authority can rely on the 'CRITICAL' and 'WARNING' alerts to mobilize resources (tugs, berth allocation, and labor) with high confidence.